In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

In [ ]:
# 1. Generate sample data
np.random.seed(42)

num_samples = 1000

# Inputs: height (150-200 cm), weight (40-100 kg), income (20k-100k)
X = np.random.rand(num_samples, 3)
X[:, 0] = X[:, 0] * 50 + 150     # height
X[:, 1] = X[:, 1] * 60 + 40      # weight
X[:, 2] = X[:, 2] * 80000 + 20000  # income